This BERT model code is taken from the google repo of ChrisMcCormickAI who had BERT Research Series playlist on YouTube. Google colab ref : https://colab.research.google.com/drive/1pTuQhug6Dhl9XalKB0zUGf4FIdYFlpcX#scrollTo=pMtmPMkBzrvs and data is taken from Kaggle NLP disaster dataset : https://www.kaggle.com/c/nlp-getting-started/data



In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from transformers import BertTokenizer
import seaborn as sns
import os

from torch.utils.data import TensorDataset, random_split

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertForSequenceClassification, AdamW, BertConfig

from transformers import get_linear_schedule_with_warmup

import random
import time
import datetime


In [2]:
#checking if GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("No GPU, using CPU instead")

GPU is available


In [3]:
os.chdir('../..')

In [4]:
os.getcwd()

'C:\\Users\\bwils\\OneDrive\\Documents\\GitHub\\WeoWater'

In [5]:
# loading the data from kaggle disaster real or not dataset
train_X_ig = pd.read_csv(r'task-2-nlp-modeling\train_test_data\instagram\train_X_ig.csv', encoding='utf-8', index_col=0)
test_X_ig = pd.read_csv(r'task-2-nlp-modeling\train_test_data\instagram\test_X_ig.csv', encoding='utf-8', index_col=0)
train_y_ig = pd.read_csv(r'task-2-nlp-modeling\train_test_data\instagram\train_y_ig.csv', encoding='utf-8', index_col=0)
test_y_ig = pd.read_csv(r'task-2-nlp-modeling\train_test_data\instagram\test_y_ig.csv', encoding='utf-8', index_col=0)

In [6]:
train_X_ig

,body
423,kaleedionne kyleweather wfaa nbcdfw There’s a ...
1024,Well first time ever we have had so much rain ...
1052,"So as I am in Oamaru, unable to return home du..."
1496,Residents of a village in Gohpur express their...
1428,LINK IN THE BIO!\n\nMumbai floods every year. ...
...,...
513,Nepal: Flash floods wreak havoc in Sindhupalch...
602,PS follow me:photoshop_xpresss #photoshop #edi...
1232,This is where I'm going next week Kerch Russ...
397,32 years ago this souvenir video was sold at U...


In [7]:
df_train = train_X_ig.merge(train_y_ig, left_index=True, right_index=True)
df_test = test_X_ig.merge(test_y_ig, left_index=True, right_index=True)

In [8]:
df_train = df_train.rename(columns={'body':'text', 'category':'target'})
df_test = df_train.rename(columns={'body':'text', 'category':'target'})

In [9]:
df_train['target'].replace({'none':0, 'flood':1}, inplace=True)

In [10]:
# category=1 implies flood relevant IG post
df_train.sort_index().head()

,text,target
2,Happy #PrideMonth from BAHA Irish Whiskey. Be ...,0
3,Figuring out these crazy hands 😆\n\n#babiesofi...,0
12,• Sewiously M0M + DAD!!? I twyin’ to watch dis...,0
29,This cute full headband and button connector f...,0
30,These cute full headbands and button connector...,0


In [11]:
df_train.columns

Index(['text', 'target'], dtype='object')

In [12]:
# BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [13]:
# Print the original IG Post
print(' Original: ', df_train.loc[2,'text'])

# Print the IG Post split into tokens.
print('Tokenized: ', tokenizer.tokenize(df_train.loc[2,'text']))

# Print the IG Post mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(df_train.loc[2, 'text'])))

 Original:  Happy #PrideMonth from BAHA Irish Whiskey. Be who you want to be. 🏳️‍🌈⠀
⠀
#Pride #LGBTQ #BAHA #IrishWhiskey #PrideMonth
Tokenized:  ['happy', '#', 'pride', '##mont', '##h', 'from', 'baha', 'irish', 'whiskey', '.', 'be', 'who', 'you', 'want', 'to', 'be', '.', '[UNK]', '[UNK]', '#', 'pride', '#', 'lgbt', '##q', '#', 'baha', '#', 'irish', '##w', '##his', '##key', '#', 'pride', '##mont', '##h']
Token IDs:  [3407, 1001, 6620, 9629, 2232, 2013, 13253, 3493, 13803, 1012, 2022, 2040, 2017, 2215, 2000, 2022, 1012, 100, 100, 1001, 6620, 1001, 12010, 4160, 1001, 13253, 1001, 3493, 2860, 24158, 14839, 1001, 6620, 9629, 2232]


In [14]:
# BERT expects certain input formats. It expects a special token at start of the sentence [CLS] and another at the end [SEP]
# Pad and truncate all sentences to a single constant length, max sentence length is 512 tokens. there  are [PAD] tokens for padding
# explicitly differentiate real tokens from padding tokens with the attention mark (1s for real tokens and 0s for padding)

# as input to BERT max length should be less than 512 and also the length of all inputs should be the same. 
max_len = 0

for post in df_train['text']:

    input_ids = tokenizer.encode(post, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))

print('Max post length: ', max_len)


Token indices sequence length is longer than the specified maximum sequence length for this model (617 > 512). Running this sequence through the model will result in indexing errors


Max post length:  617


In [15]:
# Tokenize all of the posts and map the tokens to their word IDs.
input_ids = []
attention_masks = []

for post in df_train['text']:
    encoded_dict = tokenizer.encode_plus(
                        post,                      # post to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 512,           # Pad & truncate all sentences.
                        padding = 'max_length',
                        truncation = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df_train['target'].to_numpy())

# Print sentence 0, now as a list of IDs.
print('Original: ', df_train.loc[2,'text'])
print('Token IDs:', input_ids[2])

Original:  Happy #PrideMonth from BAHA Irish Whiskey. Be who you want to be. 🏳️‍🌈⠀
⠀
#Pride #LGBTQ #BAHA #IrishWhiskey #PrideMonth
Token IDs: tensor([  101,  2061,  2004,  1045,  2572,  1999,  1051,  8067,  6820,  1010,
         4039,  2000,  2709,  2188,  2349,  2000,  9451,  1998,  2346,  8503,
         2015,  1010,  1045,  2572,  4039,  2000,  2695,  2151, 17605,  2005,
          100,  2061,  2612,  1045,  2253,  2005,  1037, 28022,  3328,  2041,
         1999,  1051,  8067,  6820,  2000,  3288,  2017,  2070, 27263,  2015,
         1997,  1996,  9882,  6652,  4348,  2023,  2237,   100,  1001,  1051,
         8067,  6820,  1001,  6652, 10441,  7849,  2226,  1001,  3181, 10441,
         7849,  2226,  1001, 19229, 12326, 18349,  8067,  6820,  1001,  1051,
         8067,  6820,  7903, 29469,  2319, 28139, 15459,  6593,  1001,  3524,
         8978, 14191,  1001, 10361,  1011,  4925,  1001,  2064,  2102, 18150,
        23393,  2063,   102,     0,     0,     0,     0,     0,     0,     0,


In [16]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks,labels)

# Create a 80-20 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

  341 training samples
   86 validation samples


In [17]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 1

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

In [18]:
# Train our classification model using BertForSequenceClassification 
# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [19]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )


In [20]:
# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [21]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [22]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))


In [23]:
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        # Perform a forward pass (evaluate the model on this training batch).
        # In PyTorch, calling `model` will in turn call the model's `forward` 
        # function and pass down the arguments. The `forward` function is 
        # documented here: 
        # https://huggingface.co/transformers/model_doc/bert.html#bertforsequenceclassification
        # The results are returned in a results object, documented here:
        # https://huggingface.co/transformers/main_classes/output.html#transformers.modeling_outputs.SequenceClassifierOutput
        # Specifically, we'll get the loss (because we provided labels) and the
        # "logits"--the model outputs prior to activation.
        result = model(b_input_ids, 
                       token_type_ids=None, 
                       attention_mask=b_input_mask, 
                       labels=b_labels,
                       return_dict=True)

        loss = result.loss
        logits = result.logits

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            result = model(b_input_ids, 
                           token_type_ids=None, 
                           attention_mask=b_input_mask,
                           labels=b_labels,
                           return_dict=True)

        # Get the loss and "logits" output by the model. The "logits" are the 
        # output values prior to applying an activation function like the 
        # softmax.
        loss = result.loss
        logits = result.logits
            
        # Accumulate the validation loss.
        total_eval_loss += loss.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()

        # Calculate the accuracy for this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        

    # Report the final accuracy for this validation run.
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


======== Epoch 1 / 2 ========
Training...


RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 2.00 GiB total capacity; 1.22 GiB already allocated; 0 bytes free; 1.29 GiB reserved in total by PyTorch)

In [ ]:
# Display floats with two decimal places.
pd.set_option('precision', 2)

# Create a DataFrame from our training statistics.
df_stats = pd.DataFrame(data=training_stats)

# Use the 'epoch' as the row index.
df_stats = df_stats.set_index('epoch')

# A hack to force the column headers to wrap.
#df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

# Display the table.
df_stats

In [ ]:


# Use plot styling from seaborn.
sns.set(style='darkgrid')

# Increase the plot size and font size.
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12,6)

# Plot the learning curve.
plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Valid. Loss'], 'g-o', label="Validation")

# Label the plot.
plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.xticks([1, 2, 3, 4])

plt.show()